# Computing thermodynamic properties

## Standard thermodynamic properties of species

Let's learn how to compute *standard thermodynamic properties of chemical species* such as:

* the standard molar Gibbs energy, $G_i^\circ$
* the standard molar Helmholtz energy, $A_i^\circ$
* the standard molar enthalpy, $H_i^\circ$
* the standard molar internal energy, $U_i^\circ$
* the standard molar entropy, $S_i^\circ$
* the standard molar volume, $V_i^\circ$
* the standard molar heat capacity (constant pressure), $C_{P,i}^\circ$
* the standard molar heat capacity (constant volume), $C_{V,i}^\circ$

Below, we use of the SUPCRTBL database to compute the standard thermodynamic properties of CO<sub></sub>(aq).

In [2]:
from reaktoro import *

db = SupcrtDatabase("supcrtbl")

CO2g    = db.species("CO2(g)")
CO2aq   = db.species("CO2(aq)")
calcite = db.species("Calcite")

We can use the method `props` in the [Species](https://reaktoro.org/api/classReaktoro_1_1Species.html) class to compute the standard thermodynamic properties of instances `CO2g`, `CO2aq`, `calcite` at 60 °C and 100 bar:

In [3]:
print("STANDARD THERMODYNAMIC PROPERTIES OF CO2(G) AT 60 °C AND 100 BAR")
print(CO2g.props(60, "C", 100, "bar"))

STANDARD THERMODYNAMIC PROPERTIES OF CO2(G) AT 60 °C AND 100 BAR
+------------------------------------------+---------+-------------+
| Property                                 |   Value |        Unit |
+------------------------------------------+---------+-------------+
| Temperature                              |  333.15 |           K |
| Pressure                                 |   1e+07 |          Pa |
| Standard Gibbs Energy                    | -401904 |       J/mol |
| Standard Enthalpy                        | -392186 |       J/mol |
| Standard Volume                          |       0 |      m3/mol |
| Standard Volume (Temperature Derivative) |       0 |  m3/(mol*K) |
| Standard Volume (Pressure Derivative)    |       0 | m3/(mol*Pa) |
| Standard Isobaric Heat Capacity          | 38.5567 |   J/(mol*K) |
| Standard Isochoric Heat Capacity         | 38.5567 |   J/(mol*K) |
| Standard Internal Energy                 | -392186 |       J/mol |
| Standard Entropy                    

Let's calculate the standard thermodynamic properties of CO<sub></sub>(aq) from 25 to 300 °C along the saturation pressure of water. Below, we build dictionary containing the following data:
* the temperatures used to evaluate thermodynamic properties
* the standard molar Gibbs energy
* the standard molar enthalpy of the species CO<sub></sub>(aq)

In [4]:
import numpy as np

temperatures = np.linspace(25.0, 300.0, 100) + 273.15  # in K

data = { "T": [], "G0": [], "H0": [] }

for T in temperatures:
    P = waterSaturationPressureWagnerPruss(T)   # in Pa
    props = CO2aq.props(T, P)                   # properties calculated in K and Pa
    data["T" ].append(float(T - 273.15))        # in °C
    data["G0"].append(float(props.G0 * 0.001))  # in kJ/mol
    data["H0"].append(float(props.H0 * 0.001))  # in kJ/mol

> **Tip:** You might also be interested in other methods for calculating the thermodynamic properties of water besides `waterSaturationPressureWagnerPruss` (which implements the water saturation pressure equation). Below, we list other methods available in [Reaktoro's API](https://reaktoro.org/api/):
>
> * `waterDensityHGK`
> * `waterDensityWagnerPruss`
> * `waterLiquidDensityHGK`
> * `waterLiquidDensityWagnerPruss`
> * `waterVaporDensityHGK`
> * `waterVaporDensityWagnerPruss`
> * `waterPressureHGK`
> * `waterPressureWagnerPruss`
> * `waterSaturationPressureWagnerPruss`
> * `waterSaturationLiquidDensityWagnerPruss`
> * `waterSaturationVapourDensityWagnerPruss`
> * `waterThermoPropsHGK`
> * `waterThermoPropsWagnerPruss`

We use the [bokeh](https://bokeh.org/) plotting library, which must be initialized to work in Jupyter Notebooks:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()

The interactive plot of standard molar Gibbs energy and standard molar enthalpy versus temperature range is buid below:

In [ ]:
hovertool = HoverTool()
hovertool.tooltips = [("Temperature", "@T °C"), ("G°,CO2(aq)", "@G0 kJ/mol"), ("H°,CO2(aq)", "@H0 kJ/mol")]

p = figure(
    title="STANDARD THERMODYNAMIC PROPERTIES OF CO2(AQ)\nALONG WATER SATURATION PRESSURE", 
    x_axis_label='TEMPERATURE [°C]', 
    y_axis_label=r"",
    sizing_mode="scale_width")

p.add_tools(hovertool)

p.line("T", "G0", source=data, legend_label="G°,CO2(aq)", line_width=5, line_cap="round", line_color="midnightblue")
p.line("T", "H0", source=data, legend_label="H°,CO2(aq)", line_width=5, line_cap="round", line_color="orange")

show(p)

**TASK**: use the PHREEQC database to collect the *standard molar internal energy* and *standard molar entropy of calcite* and plot their dependence on temperature.

In [5]:
from reaktoro import *
import numpy as np

db = PhreeqcDatabase("phreeqc.dat")

co2 = db.species("CO2")

temperatures = np.linspace(25.0, 300.0, 100) + 273.15  # in K
data = { "T": [], "U0": [], "S0": [] }

from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()

for T in temperatures:
    P = waterSaturationPressureWagnerPruss(T)   # in Pa
    props = co2.props(T, P)                     # properties calculated in K and Pa
    data["T" ].append(float(T - 273.15))        # in °C
    data["U0"].append(float(props.U0 * 0.001))  # in kJ/mol
    data["S0"].append(float(props.S0))          # in J/(mol·K)

hovertool = HoverTool()
hovertool.tooltips = [("Temperature", "@T °C"), ("U°, CO2", "@U0 kJ/mol"), ("S°, CO2 ", "@S0 kJ/(mol*K)")]

p = figure(
    title="STANDARD THERMODYNAMIC PROPERTIES OF CO2 ALONG WATER SATURATION PRESSURE",
    x_axis_label='TEMPERATURE [°C]',
    y_axis_label=r"",
    sizing_mode="scale_width",
    plot_height=300)

p.add_tools(hovertool)

p.line("T", "U0", source=data, legend_label="U°, CO2", line_width=5, line_cap="round", line_color="midnightblue")
p.line("T", "S0", source=data, legend_label="S°, CO2", line_width=5, line_cap="round", line_color="orange")

show(p)

Loading BokehJS ...

## Standard thermodynamic properties of reactions

Next, we demonstrate computation of *standard thermodynamic properties of a chemical reaction* such as:

* the equilibrium constant of reaction, $K_r$
* the standard Gibbs energy of reaction, $\Delta G_r^\circ$
* the standard Helmholtz energy of reaction, $\Delta A_r^\circ$
* the standard enthalpy of reaction, $\Delta H_r^\circ$
* the standard internal energy of reaction, $\Delta U_r^\circ$
* the standard entropy of reaction, $\Delta S_r^\circ$
* the standard volume of reaction, $\Delta V_r^\circ$
* the standard heat capacity (constant pressure) of reaction, $\Delta C_{P,r}^\circ$
* the standard heat capacity (constant volume) of reaction, $\Delta C_{V,r}^\circ$

Reaktoro computes these reaction properties using the standard thermodynamic properties of the reacting species at a given temperature and pressure.

**Example**: for reaction $2\mathrm{A}+3\mathrm{B}=\mathrm{C}$, the *standard Gibbs energy of this reaction* is computed using

$$\Delta G_{r}^{\circ}=G_{\mathrm{C}}^{\circ}-2G_{\mathrm{A}}^{\circ}-3G_{\mathrm{B}}^{\circ}.$$

This rationale (stoichiometric property contribution from product species minus the contribution of reactant species) can be applied to all other standard thermodynamic properties and every reaction.

The *equilibrium constant of the reaction* is defined as

$$\ln K_r = -\dfrac{\Delta G^{\circ}_m}{RT}$$

but it's often handled in logarithm base 10, $\lg K_r = \ln K_r/\ln 10$.

Below, we demonstrate computing of these reaction properties:

In [6]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

rxn = db.reaction("H+ + OH- = H2O")

rprops = rxn.props(25.0, "C", 1.0, "atm")
print(rprops)

+------------------------------------------------+-------------+-------------+
| Property                                       |       Value |        Unit |
+------------------------------------------------+-------------+-------------+
| Temperature                                    |    298.1500 |           K |
| Pressure                                       |      1.0132 |         bar |
| Equilibrium Constant (log base 10)             |     13.9948 |           - |
| Delta Standard Gibbs Energy                    | -79882.1698 |       J/mol |
| Delta Standard Enthalpy                        | -56358.9238 |       J/mol |
| Delta Standard Volume                          |  2.2208e-05 |      m3/mol |
| Delta Standard Volume (Temperature Derivative) |  0.0000e+00 |  m3/(mol*K) |
| Delta Standard Volume (Pressure Derivative)    |  0.0000e+00 | m3/(mol*Pa) |
| Delta Standard Isobaric Heat Capacity          |    189.6441 |   J/(mol*K) |
| Delta Standard Isochoric Heat Capacity         |  

**TASK**: consider reaction of kaolinite dissolution `Kaolinite + 6*H+ = H2O + 2*H4SiO4 + 2*Al+3` and compute its properties.

In [ ]:
rxn = db.reaction("Kaolinite + 6*H+ = H2O + 2*H4SiO4 + 2*Al+3")

rprops = rxn.props(40.0, "C", 1.0, "atm")
print(rprops)
